Install the pyBeamline library and all its dependencies.

In [ ]:
!pip install pybeamline

Ignore `pm4py` warnings.

In [2]:
import warnings
warnings.filterwarnings("ignore")

Define the MQTT endpoints for the source of the data and where the results of the mining should be published.

In [3]:
mqtt_source = {
    "broker": 'broker.emqx.io',
    "port": 1883,
    "topic": 'pybeamline/source'
}

mqtt_target = {
    "broker": 'broker.emqx.io',
    "port": 1883,
    "topic": 'pybeamline/output'
}

Define a simple `pyBeamline` pipeline that listens for events on the `mqtt_source`, performs the Heuristics Miner with Lossy Counting with Budget, converts the Heuristics Net into a Petri net (using the `pm4py` built-in conversion) and dumps the Graphviz DOT output into the `mqtt_target`.

In [ ]:
from pybeamline.sources import mqttxes_source
from pybeamline.algorithms.discovery import heuristics_miner_lossy_counting_budget
from pm4py.objects.conversion.heuristics_net import converter as conversion_factory
from pm4py.visualization.petri_net import visualizer as petri_net_visualizer
import paho.mqtt.client as mqtt


client = mqtt.Client()
client.connect(mqtt_target["broker"], mqtt_target["port"], 60)
client.loop_start()


def conversion_from_HN_to_Graphviz(heuristics_net):
  petri_net, initial_marking, final_marking = conversion_factory.apply(heuristics_net)
  gviz = petri_net_visualizer.apply(petri_net, initial_marking, final_marking)
  return str(gviz)


mqttxes_source(mqtt_source["broker"], mqtt_source["port"], mqtt_source["topic"]).pipe(
    heuristics_miner_lossy_counting_budget(model_update_frequency=4, budget=1000, dependency_threshold=0.75)
).subscribe(lambda x: client.publish(mqtt_target["topic"], conversion_from_HN_to_Graphviz(x)))

input()